In [1]:
import numpy as np

In [4]:
# Generated 100-random variables from a standard normal distribution:
data = np.random.normal(0, 1, 100)
print(data)

[ 0.17505255  0.20620789  0.30500608 -1.345659    1.26988349 -0.05326153
 -0.77408272 -1.78423104  1.03457261 -0.09011066 -0.01607513 -0.27655631
  0.55450535 -1.58663292 -1.87295086  0.03358497 -1.96071595  1.12095449
  1.77224522  1.04822616  1.16128358 -0.2737371  -1.4943713  -0.02276137
 -0.6024181   0.27239307 -1.45401341 -0.09968264 -0.04292571 -1.78268889
 -0.9741153   0.6347474  -1.33164413 -0.90615256  1.07215924  0.05828391
  0.39002911 -0.31427732 -0.16917072 -0.67209493 -0.24682637  1.40630036
  1.61433378  0.68040381  0.19303247 -0.62386412 -0.03007331  0.8748805
 -2.09659888 -0.17796994  0.38602066 -1.21938893  0.33849503  0.21160122
  0.45911113 -0.64572722 -0.30212581 -0.80471866 -1.30973712 -0.50396024
  1.76580196 -0.15045924  0.82603786  0.8656642   0.54650907 -0.31220393
 -0.444663    1.61171999 -0.1326093  -0.32753076  0.17885952  1.30714964
  0.61673433  0.83172849 -0.24182848 -0.67477327 -0.66002252 -0.13141311
 -0.5053236  -0.0155308  -1.16866425  1.14504688  1.